In [1]:
import pandas as pd
import numpy as np 

pd.set_option('display.max_columns', 300)

## Step 1: Read in hold out data, scalers, and best model

In [2]:

df = pd.read_csv('data/kc_house_data_test_features.csv', index_col=0)
print(df.shape)
df.head()


(4322, 20)


,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [3]:
import pickle

In [4]:
infile = open("pickle/model.pickle",'rb')
model = pickle.load(infile)
infile.close()

In [5]:
print(model.intercept_)
print(len(model.coef_))

-73474425.77095251
85


In [6]:
infile = open("pickle/other_info.pickle",'rb')
other_info = pickle.load(infile)
infile.close()

In [7]:
other_info

[Index(['bedrooms', 'new_bathrooms', 'sqft_living', 'sqft_lot', 'sqft_living15',
        'renovated', 'grade', 'bed_sqft', 'sqft_floor', 'month_sold',
        'waterfront', 'view', 'sqft_basement', 'sld_reno', 'wtrfr_vw',
        'condition', 'zipcode_98002', 'zipcode_98003', 'zipcode_98004',
        'zipcode_98005', 'zipcode_98006', 'zipcode_98007', 'zipcode_98008',
        'zipcode_98010', 'zipcode_98011', 'zipcode_98014', 'zipcode_98019',
        'zipcode_98022', 'zipcode_98023', 'zipcode_98024', 'zipcode_98027',
        'zipcode_98028', 'zipcode_98029', 'zipcode_98030', 'zipcode_98031',
        'zipcode_98032', 'zipcode_98033', 'zipcode_98034', 'zipcode_98038',
        'zipcode_98039', 'zipcode_98040', 'zipcode_98042', 'zipcode_98045',
        'zipcode_98052', 'zipcode_98053', 'zipcode_98055', 'zipcode_98056',
        'zipcode_98058', 'zipcode_98059', 'zipcode_98065', 'zipcode_98070',
        'zipcode_98072', 'zipcode_98074', 'zipcode_98075', 'zipcode_98077',
        'zipcode_98092

In [8]:
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [9]:
df.isna().mean()

id               0.0
date             0.0
bedrooms         0.0
bathrooms        0.0
sqft_living      0.0
sqft_lot         0.0
floors           0.0
waterfront       0.0
view             0.0
condition        0.0
grade            0.0
sqft_above       0.0
sqft_basement    0.0
yr_built         0.0
yr_renovated     0.0
zipcode          0.0
lat              0.0
long             0.0
sqft_living15    0.0
sqft_lot15       0.0
dtype: float64

## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [10]:
df["bedrooms"].replace(33,np.median(df["bedrooms"]), inplace=True)

In [11]:
df["new_bathrooms"]=np.where(df['bathrooms']<0.75, np.mean(df["bathrooms"]), df["bathrooms"])

In [12]:
df["wtrfr_vw"] = df["waterfront"]+df["view"]

In [13]:
df['renovated'] = np.where(df['yr_renovated']==0, df["yr_built"], df["yr_renovated"])
df['renovated']

0       1967
1       1967
2       2005
3       1976
4       2005
        ... 
4318    2009
4319    2014
4320    2009
4321    2004
4322    2008
Name: renovated, Length: 4322, dtype: int64

In [14]:
df["month_sold"]=pd.DatetimeIndex(df['date']).month
df["month_sold"].value_counts(ascending=True)

1     204
2     255
12    291
11    292
9     337
3     374
10    380
8     393
4     424
6     425
7     469
5     478
Name: month_sold, dtype: int64

In [15]:
df["year_sold"]=pd.DatetimeIndex(df['date']).year
df["sld_reno"]=df["renovated"]-df["year_sold"]

In [16]:
df["bed_sqft"]=df["bedrooms"]/df["sqft_living"]
df["bed_sqft"]

0       0.001762
1       0.001762
2       0.002041
3       0.002344
4       0.001413
          ...   
4318    0.001961
4319    0.001732
4320    0.001961
4321    0.001875
4322    0.001961
Name: bed_sqft, Length: 4322, dtype: float64

In [17]:
df["sqft_floor"] = df["sqft_living"]/df["floors"]

In [18]:
df = pd.get_dummies(df, columns=['zipcode'], drop_first=True)

In [19]:
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,new_bathrooms,wtrfr_vw,renovated,month_sold,year_sold,sld_reno,bed_sqft,sqft_floor,zipcode_98002,zipcode_98003,zipcode_98004,zipcode_98005,zipcode_98006,zipcode_98007,zipcode_98008,zipcode_98010,zipcode_98011,zipcode_98014,zipcode_98019,zipcode_98022,zipcode_98023,zipcode_98024,zipcode_98027,zipcode_98028,zipcode_98029,zipcode_98030,zipcode_98031,zipcode_98032,zipcode_98033,zipcode_98034,zipcode_98038,zipcode_98039,zipcode_98040,zipcode_98042,zipcode_98045,zipcode_98052,zipcode_98053,zipcode_98055,zipcode_98056,zipcode_98058,zipcode_98059,zipcode_98065,zipcode_98070,zipcode_98072,zipcode_98074,zipcode_98075,zipcode_98077,zipcode_98092,zipcode_98102,zipcode_98103,zipcode_98105,zipcode_98106,zipcode_98107,zipcode_98108,zipcode_98109,zipcode_98112,zipcode_98115,zipcode_98116,zipcode_98117,zipcode_98118,zipcode_98119,zipcode_98122,zipcode_98125,zipcode_98126,zipcode_98133,zipcode_98136,zipcode_98144,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,47.7089,-122.241,2020,10918,2.50,0,1967,8,2014,-47,0.001762,2270.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,47.7089,-122.241,2020,10918,2.50,0,1967,2,2015,-48,0.001762,2270.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,47.5472,-121.998,1470,1576,2.50,0,2005,11,2014,-9,0.002041,735.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,47.7427,-122.071,1160,10565,1.75,0,1976,12,2014,-38,0.002344,1280.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,47.4863,-122.140,2830,7916,2.75,0,2005,1,2015,-10,0.001413,1415.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
df.shape

(4322, 96)

## Step 3: Predict the holdout set

In [20]:
list(other_info)

[Index(['bedrooms', 'new_bathrooms', 'sqft_living', 'sqft_lot', 'sqft_living15',
        'renovated', 'grade', 'bed_sqft', 'sqft_floor', 'month_sold',
        'waterfront', 'view', 'sqft_basement', 'sld_reno', 'wtrfr_vw',
        'condition', 'zipcode_98002', 'zipcode_98003', 'zipcode_98004',
        'zipcode_98005', 'zipcode_98006', 'zipcode_98007', 'zipcode_98008',
        'zipcode_98010', 'zipcode_98011', 'zipcode_98014', 'zipcode_98019',
        'zipcode_98022', 'zipcode_98023', 'zipcode_98024', 'zipcode_98027',
        'zipcode_98028', 'zipcode_98029', 'zipcode_98030', 'zipcode_98031',
        'zipcode_98032', 'zipcode_98033', 'zipcode_98034', 'zipcode_98038',
        'zipcode_98039', 'zipcode_98040', 'zipcode_98042', 'zipcode_98045',
        'zipcode_98052', 'zipcode_98053', 'zipcode_98055', 'zipcode_98056',
        'zipcode_98058', 'zipcode_98059', 'zipcode_98065', 'zipcode_98070',
        'zipcode_98072', 'zipcode_98074', 'zipcode_98075', 'zipcode_98077',
        'zipcode_98092

In [21]:
final_columns = list(other_info[0])

In [22]:
df[final_columns]

,bedrooms,new_bathrooms,sqft_living,sqft_lot,sqft_living15,renovated,grade,bed_sqft,sqft_floor,month_sold,waterfront,view,sqft_basement,sld_reno,wtrfr_vw,condition,zipcode_98002,zipcode_98003,zipcode_98004,zipcode_98005,zipcode_98006,zipcode_98007,zipcode_98008,zipcode_98010,zipcode_98011,zipcode_98014,zipcode_98019,zipcode_98022,zipcode_98023,zipcode_98024,zipcode_98027,zipcode_98028,zipcode_98029,zipcode_98030,zipcode_98031,zipcode_98032,zipcode_98033,zipcode_98034,zipcode_98038,zipcode_98039,zipcode_98040,zipcode_98042,zipcode_98045,zipcode_98052,zipcode_98053,zipcode_98055,zipcode_98056,zipcode_98058,zipcode_98059,zipcode_98065,zipcode_98070,zipcode_98072,zipcode_98074,zipcode_98075,zipcode_98077,zipcode_98092,zipcode_98102,zipcode_98103,zipcode_98105,zipcode_98106,zipcode_98107,zipcode_98108,zipcode_98109,zipcode_98112,zipcode_98115,zipcode_98116,zipcode_98117,zipcode_98118,zipcode_98119,zipcode_98122,zipcode_98125,zipcode_98126,zipcode_98133,zipcode_98136,zipcode_98144,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
0,4,2.50,2270,11500,2020,1967,8,0.001762,2270.0,8,0,0,730,-47,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,4,2.50,2270,11500,2020,1967,8,0.001762,2270.0,2,0,0,730,-48,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,2.50,1470,1779,1470,2005,8,0.002041,735.0,11,0,0,310,-9,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,1.75,1280,16200,1160,1976,8,0.002344,1280.0,12,0,0,250,-38,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,2.75,2830,8126,2830,2005,8,0.001413,1415.0,1,0,0,0,-10,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,3,2.50,1530,1131,1530,2009,8,0.001961,510.0,5,0,0,0,-5,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4319,4,2.50,2310,5813,1830,2014,8,0.001732,1155.0,2,0,0,0,-1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4320,2,0.75,1020,1350,1020,2009,7,0.001961,510.0,6,0,0,0,-5,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4321,3,2.50,1600,2388,1410,2004,8,0.001875,800.0,1,0,0,0,-11,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
final_answers = model.predict(df[final_columns])

In [24]:
final_answers

array([524697.86275354, 551387.67961775, 411094.14476718, ...,
       329503.89105913, 408586.06574629, 336505.05518167])

In [25]:
len(final_answers)

4322

In [26]:
df_final = pd.DataFrame(final_answers)

In [31]:
df_final

,0
0,524697.862754
1,551387.679618
2,411094.144767
3,303834.461122
4,567961.441684
...,...
4317,551155.269727
4318,450087.533127
4319,329503.891059
4320,408586.065746


## Step 4: Export your predictions

In [28]:
df_final.to_csv('data/housing_preds_vaneeza_ahmad.csv')